In [2]:
import pandas as pd
csv_file="final.csv"
df=pd.read_csv(csv_file)
df

,Unnamed: 0,Series_Title,Released_Year,Certificate,Overview,Director,Star1
0,0,The Shawshank Redemption,1994,A,Two imprisoned men bond over a number of years...,Frank Darabont,Tim Robbins
1,1,The Godfather,1972,A,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon Brando
2,2,The Dark Knight,2008,UA,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian Bale
3,3,The Godfather: Part II,1974,A,The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al Pacino
4,4,12 Angry Men,1957,U,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,Henry Fonda
...,...,...,...,...,...,...,...
995,995,Breakfast at Tiffany's,1961,A,A young New York socialite becomes interested ...,Blake Edwards,Audrey Hepburn
996,996,Giant,1956,G,Sprawling epic covering the life of a Texas ca...,George Stevens,Elizabeth Taylor
997,997,From Here to Eternity,1953,Passed,"In Hawaii in 1941, a private is cruelly punish...",Fred Zinnemann,Burt Lancaster
998,998,Lifeboat,1944,NaN,Several survivors of a torpedoed merchant ship...,Alfred Hitchcock,Tallulah Bankhead


In [3]:
from py2neo import Graph,Node,Relationship
import csv
graph = Graph("bolt://localhost:7687", auth=("neo4j", "sanjay123"))

In [4]:
def clear_graph():
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    graph.run(query)
    print("Graph has been cleared...")

In [5]:
clear_graph()

Graph has been cleared...


In [6]:
# Read the CSV file and create nodes and relationships
with open(csv_file, "r",encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        index,Series_Title, Released_Year,Certificate,Overview,Director,Star1= row

        movie_node=Node("Movie",name=Series_Title)
        graph.merge(movie_node,"Movie","name")
        
        year_node=Node("Year",name=Released_Year)
        graph.merge(year_node,"Year","name")
        released_in_relationship=Relationship(movie_node, "RELEASED_IN", year_node)
        graph.create(released_in_relationship)
        
        cert_node = Node("Cert", name=Certificate)
        graph.merge(cert_node,"Cert","name")
        has_certificate_relationship=Relationship(movie_node, "HAS_CERTIFICATE", cert_node)
        graph.create(has_certificate_relationship)
        
        plot_node = Node("plot", name=Overview)
        graph.merge(plot_node,"plot","name")
        has_plot_relationship=Relationship(movie_node, "HAS_PLOT", plot_node)
        graph.create(has_plot_relationship)
        
        director_node = Node("director", name=Director)
        graph.merge(director_node,"director","name")
        has_director_relationship=Relationship(movie_node, "HAS_DIRECTOR", director_node)
        graph.create(has_director_relationship)
        
        star_node = Node("star", name=Star1)
        graph.merge(star_node,"star","name")
        has_acted_relationship=Relationship(star_node, "HAS_ACTED", movie_node)
        graph.create(has_acted_relationship)

In [7]:
label="Movie"
query = f"""
MATCH (n:{label})
RETURN COUNT(n) AS nodeCount
"""

# Execute the query and fetch the result
result = graph.run(query).data()
print(result)

[{'nodeCount': 999}]
